In [2]:
import os
import pandas as pd
import re
import requests
import zipfile
import math
from itertools import compress
from bs4 import BeautifulSoup
from pprint import pprint
from pymongo.errors import BulkWriteError

from unidecode import unidecode

In [5]:
## PRE PROCESSING
len(lista)

338

In [4]:
lista = os.listdir('../data/consulta_cand')
lista = [e for e in lista if ('.txt' in e) or ('.csv' in e)]
consulta_cand_csvs = lista

In [4]:
def unifica_na(value):
    if (type(value) == 'numpy.ndarray'):
        return('#NULO#')
    if (type(value) == float) or (type(value) == int):
        if (math.isnan(value)) or value < 0:
            return "NULO"
        else:
            return value
    elif ('#ne' in value.lower()) or\
    ('#ni' in value.lower()) or\
    ('#nulo' in value.lower()) or\
    ('-1' in value.lower()) or\
    ('-4' in value.lower()) or\
    ('-3' in value.lower()):
        #print('na')
        return "NULO"
    else:
        value = unidecode(value)
        return value

# remove null keys    
def remove_null_keys(obj):
    dict_keys = list(obj.keys())
    del_bool_array = [f == "#NULO#" for f in obj.values()]

    del_elements = list(compress(dict_keys, del_bool_array))

    for f in del_elements:
        del obj[f]
    return obj


In [5]:
def row2query(row):
    col_names = list(row.columns)
    col_names = ['"{}"'.format(e) for e in col_names]
    col_names = ', '.join(col_names)
    col_names
    values = list(row.loc[0].values)
    values = ', '.join(values)

    query = 'INSERT INTO CANDIDATURAS ({}) VALUES ({});'.format(col_names, values)
    return query


def row2query(row):
    col_names = list(row.index)
    col_names = ['`{}`'.format(e) for e in col_names]
    col_names = ', '.join(col_names)
    
    values = ['%s' for e in row.index]
    values = ', '.join(values)

    query = 'INSERT INTO CANDIDATURAS ({}) VALUES ({});'.format(col_names, values)

    return query

# MySQL

In [12]:
import pymysql.cursors
import pymysql

# MySQL CONNECTION
connection = pymysql.connect(host='localhost',
                            user='root',
                            password='banana123',
                            db='OUTSIDERS',
                            charset='utf8')


# iterate over files and shove them up to the databases
for i in range(0, len(consulta_cand_csvs)):
    # tracking
    print("{}/{}".format(i, len(consulta_cand_csvs)), consulta_cand_csvs[i])

    # var setup
    ano = re.search('[0-9]{4}', consulta_cand_csvs[i]).group(0)
    ano = int(ano)

    # chose loading method dependint on file format
    if '.txt' in consulta_cand_csvs[i]:
        skiprows = 0
    elif '.csv' in consulta_cand_csvs[i]:
        skiprows = 1
    data = pd.read_csv('../data/consulta_cand/' + consulta_cand_csvs[i], encoding='latin1', sep=';', header=None, skiprows=skiprows, dtype=object)
    data = data.drop_duplicates()
    # SET CORRECT COL NAMES
    col_names = {
        '1994-2010':'DATA_GERACAO HORA_GERACAO ANO_ELEICAO NUMERO_TURNO DESCRICAO_ELEICAO SIGLA_UF SIGLA_UE DESCRICAO_UE CODIGO_CARGO DESCRICAO_CARGO NOME_CANDIDATO SEQUENCIAL_CANDIDATO NUMERO_CANDIDATO NUMERO_CPF_CANDIDATO NOME_URNA_CANDIDATO CODIGO_SITUACAO_CANDIDATURA DESCRICAO_SITUACAO_CANDIDATURA NUMERO_PARTIDO SIGLA_PARTIDO NOME_PARTIDO CODIGO_LEGENDA SIGLA_LEGENDA COMPOSICAO_LEGENDA NOME_LEGENDA CODIGO_OCUPACAO DESCRICAO_OCUPACAO DATA_NASCIMENTO NUMERO_TITULO_ELEITORAL_CANDIDATO IDADE_DATA_ELEICAO CODIGO_SEXO DESCRICAO_SEXO CODIGO_GRAU_INSTRUCAO DESCRICAP_GRAU_INSTRUCAO CODIGO_ESTADO_CIVIL DESCRICAO_ESTADO_CIVIL CODIGO_NACIONALIDADE DESCRICAO_NACIONALIDADE SIGLA_UF_NASCIMENTO CODIGO_MUNICIPIO_NASCIMENTO NOME_MUNICIPIO_NASCIMENTO DESPESA_MAX_CAMPANHA CODIGO_SITUACAO_TOTAL_TURNO DESCRICAO_SITUACAO_TOTAL_TURNO'.split(' '),
        '2012':'DATA_GERACAO HORA_GERACAO ANO_ELEICAO NUMERO_TURNO DESCRICAO_ELEICAO SIGLA_UF SIGLA_UE DESCRICAO_UE CODIGO_CARGO DESCRICAO_CARGO NOME_CANDIDATO SEQUENCIAL_CANDIDATO NUMERO_CANDIDATO NUMERO_CPF_CANDIDATO NOME_URNA_CANDIDATO CODIGO_SITUACAO_CANDIDATURA DESCRICAO_SITUACAO_CANDIDATURA NUMERO_PARTIDO SIGLA_PARTIDO NOME_PARTIDO CODIGO_LEGENDA SIGLA_LEGENDA COMPOSICAO_LEGENDA NOME_LEGENDA CODIGO_OCUPACAO DESCRICAO_OCUPACAO DATA_NASCIMENTO NUMERO_TITULO_ELEITORAL_CANDIDATO IDADE_DATA_ELEICAO CODIGO_SEXO DESCRICAO_SEXO CODIGO_GRAU_INSTRUCAO DESCRICAP_GRAU_INSTRUCAO CODIGO_ESTADO_CIVIL DESCRICAO_ESTADO_CIVIL CODIGO_NACIONALIDADE DESCRICAO_NACIONALIDADE SIGLA_UF_NASCIMENTO CODIGO_MUNICIPIO_NASCIMENTO NOME_MUNICIPIO_NASCIMENTO DESPESA_MAX_CAMPANHA CODIGO_SITUACAO_TOTAL_TURNO DESCRICAO_SITUACAO_TOTAL_TURNO NOME_EMAIL'.split(' '),
        '2014':'DATA_GERACAO HORA_GERACAO ANO_ELEICAO CODIGO_TIPO_ELEICAO NOME_TIPO_ELEICAO NUMERO_TURNO CODIGO_ELEICAO DESCRICAO_ELEICAO DATA_ELEICAO TIPO_ABRANGENCIA SIGLA_UF SIGLA_UE NOME_UE CODIGO_CARGO DESCRICAO_CARGO SEQUENCIAL_CANDIDATO NUMERO_CANDIDATO NOME_CANDIDATO NOME_URNA_CANDIDATO NOME_SOCIAL_CANDIDATO NUMERO_CPF_CANDIDATO NOME_EMAIL CODIGO_SITUACAO_CANDIDATURA DESCRICAO_SITUACAO_CANDIDATURA CODIGO_DETALHE_SITUACAO_CANDIDATURA DESCRICAO_DETALHE_SITUACAO_CANDIDATURA TIPO_AGREMIACAO NUMERO_PARTIDO SIGLA_PARTIDO NOME_PARTIDO SEQUENCIAL_COLIGACAO NOME_COLIGACAO DESCRICAO_COMPOSICAO_COLIGACAO CODIGO_NACIONALIDADE DESCRICAO_NACIONALIDADE SIGLA_UF_NASCIMENTO CODIGO_MUNICIPIO_NASCIMENTO NOME_MUNICIPIO_NASCIMENTO DATA_NASCIMENTO IDADE_DATA_POSSE NUMERO_TITULO_ELEITORAL_CANDIDATO CODIGO_SEXO DESCRICAO_SEXO CODIGO_GRAU_INSTRUCAO DESCRICAO_GRAU_INSTRUCAO CODIGO_ESTADO_CIVIL DESCRICAO_ESTADO_CIVIL CODIGO_COR_RACA DESCRICAO_COR_RACA CODIGO_OCUPACAO DESCRICAO_OCUPACAO DESPESA_MAX_CAMPANHA CODIGO_SITUACAO_TOTAL_TURNO DESCRICAO_SITUACAO_TOTAL_TURNO SITUACAO_REELEICAO SITUACAO_DECLARAR_BENS NUMERO_PROTOCOLO_CANDIDATURA NUMERO_PROCESSO'.split(' '),
        '2016':'DATA_GERACAO HORA_GERACAO ANO_ELEICAO CODIGO_TIPO_ELEICAO NOME_TIPO_ELEICAO NUMERO_TURNO CODIGO_ELEICAO DESCRICAO_ELEICAO DATA_ELEICAO TIPO_ABRANGENCIA SIGLA_UF SIGLA_UE NOME_UE CODIGO_CARGO DESCRICAO_CARGO SEQUENCIAL_CANDIDATO NUMERO_CANDIDATO NOME_CANDIDATO NOME_URNA_CANDIDATO NOME_SOCIAL_CANDIDATO NUMERO_CPF_CANDIDATO NOME_EMAIL CODIGO_SITUACAO_CANDIDATURA DESCRICAO_SITUACAO_CANDIDATURA CODIGO_DETALHE_SITUACAO_CANDIDATURA DESCRICAO_DETALHE_SITUACAO_CANDIDATURA TIPO_AGREMIACAO NUMERO_PARTIDO SIGLA_PARTIDO NOME_PARTIDO SEQUENCIAL_COLIGACAO NOME_COLIGACAO DESCRICAO_COMPOSICAO_COLIGACAO CODIGO_NACIONALIDADE DESCRICAO_NACIONALIDADE SIGLA_UF_NASCIMENTO CODIGO_MUNICIPIO_NASCIMENTO NOME_MUNICIPIO_NASCIMENTO DATA_NASCIMENTO IDADE_DATA_POSSE NUMERO_TITULO_ELEITORAL_CANDIDATO CODIGO_SEXO DESCRICAO_SEXO CODIGO_GRAU_INSTRUCAO DESCRICAO_GRAU_INSTRUCAO CODIGO_ESTADO_CIVIL DESCRICAO_ESTADO_CIVIL CODIGO_COR_RACA DESCRICAO_COR_RACA CODIGO_OCUPACAO DESCRICAO_OCUPACAO DESPESA_MAX_CAMPANHA CODIGO_SITUACAO_TOTAL_TURNO DESCRICAO_SITUACAO_TOTAL_TURNO SITUACAO_REELEICAO SITUACAO_DECLARAR_BENS NUMERO_PROTOCOLO_CANDIDATURA NUMERO_PROCESSO'.split(' '),
        '2018':'DATA_GERACAO HORA_GERACAO ANO_ELEICAO CODIGO_TIPO_ELEICAO NOME_TIPO_ELEICAO NUMERO_TURNO CODIGO_ELEICAO DESCRICAO_ELEICAO DATA_ELEICAO TIPO_ABRANGENCIA SIGLA_UF SIGLA_UE NOME_UE CODIGO_CARGO DESCRICAO_CARGO SEQUENCIAL_CANDIDATO NUMERO_CANDIDATO NOME_CANDIDATO NOME_URNA_CANDIDATO NOME_SOCIAL_CANDIDATO NUMERO_CPF_CANDIDATO NOME_EMAIL CODIGO_SITUACAO_CANDIDATURA DESCRICAO_SITUACAO_CANDIDATURA CODIGO_DETALHE_SITUACAO_CANDIDATURA DESCRICAO_DETALHE_SITUACAO_CANDIDATURA TIPO_AGREMIACAO NUMERO_PARTIDO SIGLA_PARTIDO NOME_PARTIDO SEQUENCIAL_COLIGACAO NOME_COLIGACAO DESCRICAO_COMPOSICAO_COLIGACAO CODIGO_NACIONALIDADE DESCRICAO_NACIONALIDADE SIGLA_UF_NASCIMENTO CODIGO_MUNICIPIO_NASCIMENTO NOME_MUNICIPIO_NASCIMENTO DATA_NASCIMENTO IDADE_DATA_POSSE NUMERO_TITULO_ELEITORAL_CANDIDATO CODIGO_SEXO DESCRICAO_SEXO CODIGO_GRAU_INSTRUCAO DESCRICAO_GRAU_INSTRUCAO CODIGO_ESTADO_CIVIL DESCRICAO_ESTADO_CIVIL CODIGO_COR_RACA DESCRICAO_COR_RACA CODIGO_OCUPACAO DESCRICAO_OCUPACAO DESPESA_MAX_CAMPANHA CODIGO_SITUACAO_TOTAL_TURNO DESCRICAO_SITUACAO_TOTAL_TURNO SITUACAO_REELEICAO SITUACAO_DECLARAR_BENS NUMERO_PROTOCOLO_CANDIDATURA NUMERO_PROCESSO'.split(' ')
    }
    if (ano >= 1994 and ano <= 2010):
        if consulta_cand_csvs[i] == 'consulta_cand_1994_BR.txt':
            data = data.iloc[:,:22].join(data.iloc[:,23:])
        data.columns = col_names['1994-2010']
    elif(ano == 2012):
        data.columns = col_names['2012']
    elif(ano ==  2014):
        data.columns = col_names['2014']
    elif(ano ==  2016):
        data.columns = col_names['2016']
    elif (ano == 2018):
        data.columns = col_names['2018']

    # SET ID COL
    #data['ID'] = ['{}_{}_{}_{}_{}_{}_{}_{}'.format(data.loc[j, 'SEQUENCIAL_CANDIDATO'], data.loc[j, 'CODIGO_SITUACAO_CANDIDATURA'], data.loc[j, 'NUMERO_TITULO_ELEITORAL_CANDIDATO'], data.loc[j, 'ANO_ELEICAO'], data.loc[j, 'SIGLA_UF'], data.loc[j, 'CODIGO_SITUACAO_TOTAL_TURNO'], data.loc[j, 'NUMERO_TURNO'], data.loc[j, 'CODIGO_CARGO']) for j in data.index]

    # remove duplicates
    print("len: ", len(data))
        # turns all nulls in the same value
    data = data.applymap(unifica_na)
    #data = data[[i != 'nan_nan_nan_nan_nan_nan_nan_nan' for i in data['ID']]]
    #break
    
    try:
        with connection.cursor() as cursor:
            # Create a new record
            for j in data.index:
                
                query = row2query(data.loc[j])
                values = tuple(data.loc[j].values)
                if len(set(values)) < 5:
                    continue
                else:
                    cursor.execute(query, tuple(data.loc[j].values))

        # connection is not autocommit by default. So you must commit to save
        # your changes.
        connection.commit()
    
    finally:
        pass
connection.close()
    

0/338 consulta_cand_2012_RN.txt
len:  8891
1/338 consulta_cand_2018_RS.csv
len:  1344
2/338 consulta_cand_2008_MG.txt
len:  59850
3/338 consulta_cand_2018_MA.csv
len:  796
4/338 consulta_cand_2008_ES.txt
len:  6956
5/338 consulta_cand_2016_CE.csv
len:  14954
6/338 consulta_cand_2002_RR.txt
len:  595
7/338 consulta_cand_2014_TO.csv
len:  395
8/338 consulta_cand_1998_RS.txt
len:  657
9/338 consulta_cand_2008_SC.txt
len:  14393
10/338 consulta_cand_2012_MS.txt
len:  7127
11/338 consulta_cand_2004_MA.txt
len:  15642
12/338 consulta_cand_1998_PI.txt
len:  274
13/338 consulta_cand_2016_RN.csv
len:  9173
14/338 consulta_cand_2000_AM.txt
len:  5818
15/338 consulta_cand_2010_BA.txt
len:  1020
16/338 consulta_cand_2000_PI.txt
len:  7528
17/338 consulta_cand_2010_CE.txt
len:  807
18/338 consulta_cand_2006_PE.txt
len:  802
19/338 consulta_cand_2016_PI.csv
len:  10202
20/338 consulta_cand_1994_MS.txt
len:  166
21/338 consulta_cand_2006_RR.txt
len:  529
22/338 consulta_cand_2000_PA.txt
len:  13628
2

186/338 consulta_cand_2002_PI.txt
len:  345
187/338 consulta_cand_2000_RJ.txt
len:  17774
188/338 consulta_cand_2000_PE.txt
len:  13063
189/338 consulta_cand_2002_AC.txt
len:  469
190/338 consulta_cand_2010_MG.txt
len:  1789
191/338 consulta_cand_2014_MT.csv
len:  485
192/338 consulta_cand_2002_BA.txt
len:  787
193/338 consulta_cand_2014_SC.csv
len:  676
194/338 consulta_cand_2012_AC.txt
len:  2343
195/338 consulta_cand_2006_MG.txt
len:  1630
196/338 consulta_cand_2008_MA.txt
len:  15718
197/338 consulta_cand_1998_AP.txt
len:  325
198/338 consulta_cand_1994_BR.txt
len:  16
199/338 consulta_cand_2014_SE.csv
len:  303
200/338 consulta_cand_2000_AP.txt
len:  1412
201/338 consulta_cand_2010_RN.txt
len:  249
202/338 consulta_cand_2000_RR.txt
len:  1239
203/338 consulta_cand_2018_PA.csv
len:  903
204/338 consulta_cand_2008_AC.txt
len:  1520
205/338 consulta_cand_2012_RS.txt
len:  29111
206/338 consulta_cand_2012_TO.txt
len:  7715
207/338 consulta_cand_2012_MG.txt
len:  75126
208/338 consulta

In [35]:
CREATE TABLE CANDIDATURAS(
SIGLA_LEGENDA varchar(200),
NOME_CANDIDATO varchar(200),
DESCRICAO_DETALHE_SITUACAO_CANDIDATURA varchar(200),
SIGLA_UE varchar(200),
CODIGO_ELEICAO varchar(200),
NOME_COLIGACAO varchar(200),
DESCRICAO_SITUACAO_TOTAL_TURNO varchar(200),
CODIGO_COR_RACA varchar(200),
NUMERO_PROTOCOLO_CANDIDATURA varchar(200),
CODIGO_LEGENDA varchar(200),
DESCRICAO_OCUPACAO varchar(200),
ANO_ELEICAO int,
DATA_NASCIMENTO varchar(200),
CODIGO_NACIONALIDADE varchar(200),
SIGLA_PARTIDO varchar(200),
NUMERO_TITULO_ELEITORAL_CANDIDATO varchar(200),
DESCRICAO_ELEICAO varchar(200),
DESCRICAO_CARGO varchar(200),
NOME_SOCIAL_CANDIDATO varchar(200),
NOME_LEGENDA varchar(200),
SITUACAO_REELEICAO varchar(200),
DESCRICAO_SITUACAO_CANDIDATURA varchar(200),
DESCRICAO_SEXO varchar(200),
CODIGO_ESTADO_CIVIL varchar(200),
NUMERO_CANDIDATO varchar(200),
NUMERO_PARTIDO varchar(200),
DESCRICAP_GRAU_INSTRUCAO varchar(200),
CODIGO_MUNICIPIO_NASCIMENTO varchar(200),
NUMERO_CPF_CANDIDATO varchar(200),
NOME_EMAIL varchar(200),
CODIGO_TIPO_ELEICAO varchar(200),
TIPO_ABRANGENCIA varchar(200),
COMPOSICAO_LEGENDA varchar(200),
DESCRICAO_UE varchar(200),
NOME_PARTIDO varchar(200),
IDADE_DATA_ELEICAO varchar(200),
NOME_UE varchar(200),
CODIGO_DETALHE_SITUACAO_CANDIDATURA varchar(200),
DESCRICAO_ESTADO_CIVIL varchar(200),
SITUACAO_DECLARAR_BENS varchar(200),
SIGLA_UF_NASCIMENTO varchar(200),
DESPESA_MAX_CAMPANHA varchar(200),
SEQUENCIAL_COLIGACAO varchar(200),
IDADE_DATA_POSSE varchar(200),
SEQUENCIAL_CANDIDATO varchar(200),
HORA_GERACAO varchar(200),
NOME_TIPO_ELEICAO varchar(200),
CODIGO_GRAU_INSTRUCAO varchar(200),
CODIGO_SEXO varchar(200),
CODIGO_SITUACAO_TOTAL_TURNO varchar(200),
NUMERO_TURNO varchar(200),
CODIGO_SITUACAO_CANDIDATURA varchar(200),
CODIGO_CARGO varchar(200),
DESCRICAO_GRAU_INSTRUCAO varchar(200),
SIGLA_UF varchar(200),
DESCRICAO_COR_RACA varchar(200),
NOME_URNA_CANDIDATO varchar(200),
NUMERO_PROCESSO varchar(200),
DESCRICAO_COMPOSICAO_COLIGACAO varchar(200),
CODIGO_OCUPACAO varchar(200),
NOME_MUNICIPIO_NASCIMENTO varchar(200),
TIPO_AGREMIACAO varchar(200),
DESCRICAO_NACIONALIDADE varchar(200),
DATA_ELEICAO varchar(200),
DATA_GERACAO varchar(200)
);

SyntaxError: invalid syntax (<ipython-input-35-2859770bc1ce>, line 1)

In [8]:
#data = pd.read_csv('../data/consulta_cand/consulta_cand_1994_BR.txt', sep=';', encoding='latin1')
len(data.columns)
j = 2
for i in range(len(data.loc[j,:].values)):
    print((i, data.loc[j,:].values[i]))

(0, '26/04/2018')
(1, '19:31:27')
(2, '2008')
(3, '1')
(4, 'Eleições 2008')
(5, 'MA')
(6, '07013')
(7, 'AFONSO CUNHA')
(8, '12')
(9, 'VICE-PREFEITO')
(10, 'ANTONIO MARIA CRISPIM')
(11, '12022')
(12, '65')
(13, '07587643320')
(14, 'ANTONIO MARIA')
(15, '2')
(16, 'DEFERIDO')
(17, '10')
(18, 'PRB')
(19, 'PARTIDO REPUBLICANO BRASILEIRO')
(20, '1181')
(21, '#NE#')
(22, 'AFONSO CUNHA CONTINUA CRESCENDO')
(23, '601')
(24, 'AGRICULTOR')
(25, '01/05/1935')
(26, '007223001155')
(27, '-1')
(28, '2')
(29, 'MASCULINO')
(30, '3')
(31, 'ENSINO FUNDAMENTAL INCOMPLETO')
(32, '3')
(33, 'CASADO(A)')
(34, '1')
(35, 'BRASILEIRA NATA')
(36, 'PI')
(37, '-1')
(38, 'UNIÃO')
(39, '-1')
(40, '-1')
(41, '#NULO#')


In [85]:
data.iloc[:,23:]

,#NULO.3,#NULO.4,0,##VERIFICAR BASE 1994##,21/12/47,5399180914,-1.3,2,MASCULINO,8,...,3,##VERIFICAR BASE 1994##.2,1.2,##VERIFICAR BASE 1994##.3,#NE.1,-1.4,#NE.2,-1.5,4,NAO ELEITO
0,#NULO,#NULO,6,##VERIFICAR BASE 1994##,22/01/22,18655100361,-1,2,MASCULINO,8.0,...,5,##VERIFICAR BASE 1994##,1,##VERIFICAR BASE 1994##,#NE,-1,#NE,-1,4,NAO ELEITO
1,PT/PSTU/PPS/PSB/PV,FRENTE BRASIL POPULAR PELA CIDADANIA,195,##VERIFICAR BASE 1994##,06/10/45,122418060191,-1,2,MASCULINO,4.0,...,3,##VERIFICAR BASE 1994##,1,##VERIFICAR BASE 1994##,#NE,-1,#NE,-1,4,NAO ELEITO
2,PMDB/PSD,DESENVOLVIMENTO DO BRASIL,57,##VERIFICAR BASE 1994##,18/08/38,16813380132,-1,2,MASCULINO,8.0,...,3,##VERIFICAR BASE 1994##,1,##VERIFICAR BASE 1994##,#NE,-1,#NE,-1,4,NAO ELEITO
3,#NULO,#NULO,253,##VERIFICAR BASE 1994##,09/06/32,17781030353,-1,2,MASCULINO,NaN,...,3,##VERIFICAR BASE 1994##,1,##VERIFICAR BASE 1994##,#NE,-1,#NE,-1,4,NAO ELEITO
4,#NULO,#NULO,296,##VERIFICAR BASE 1994##,15/11/55,40904900531,-1,2,MASCULINO,8.0,...,3,##VERIFICAR BASE 1994##,1,##VERIFICAR BASE 1994##,#NE,-1,#NE,-1,4,NAO ELEITO
5,PTB/PFL/PSDB,"UNIAO, TRABALHO E PROGRESSO",86,##VERIFICAR BASE 1994##,18/06/31,95637000116,-1,2,MASCULINO,8.0,...,3,##VERIFICAR BASE 1994##,1,##VERIFICAR BASE 1994##,#NE,-1,#NE,-1,1,ELEITO
6,#NULO,#NULO,37,##VERIFICAR BASE 1994##,05/11/38,16352040310,-1,2,MASCULINO,8.0,...,3,##VERIFICAR BASE 1994##,1,##VERIFICAR BASE 1994##,#NE,-1,#NE,-1,4,NAO ELEITO
7,#NULO,#NULO,296,##VERIFICAR BASE 1994##,01/03/40,3365571139,-1,4,FEMININO,6.0,...,3,##VERIFICAR BASE 1994##,1,##VERIFICAR BASE 1994##,#NE,-1,#NE,-1,-1,#NULO#
8,#NULO,#NULO,87,##VERIFICAR BASE 1994##,26/10/22,18171980329,-1,2,MASCULINO,8.0,...,9,##VERIFICAR BASE 1994##,1,##VERIFICAR BASE 1994##,#NE,-1,#NE,-1,-1,#NULO#
9,#NULO,FRENTE BRASIL POPULAR PELA CIDADANIA,52,##VERIFICAR BASE 1994##,13/05/54,87017320167,-1,2,MASCULINO,8.0,...,3,##VERIFICAR BASE 1994##,1,##VERIFICAR BASE 1994##,#NE,-1,#NE,-1,-1,#NULO#


In [86]:
data.iloc[:,:22]#==data.iloc[:,23]

,21/11/2018,15:22:40,1994,1,ELEICOES 1994,BR,BR.1,BRASIL,1.1,PRESIDENTE,...,#NULO,ESPERIDIAO AMIN,-1.1,#NE,11.1,PPR,PARTIDO PROGRESSISTA REFORMADOR,-1.2,#NULO.1,#NULO.2
0,21/11/2018,15:22:40,1994,1,ELEICOES 1994,BR,BR,BRASIL,1,PRESIDENTE,...,#NULO,BRIZOLA,-1,#NE,12,PDT,PARTIDO DEMOCRATICO TRABALHISTA,-1,#NULO,#NULO
1,21/11/2018,15:22:40,1994,1,ELEICOES 1994,BR,BR,BRASIL,1,PRESIDENTE,...,#NULO,LULA,-1,#NE,13,PT,PARTIDO DOS TRABALHADORES,-1,FBPPC,PT/PSTU/PPS/PSB/PV
2,21/11/2018,15:22:40,1994,1,ELEICOES 1994,BR,BR,BRASIL,1,PRESIDENTE,...,#NULO,ORESTES QUERCIA,-1,#NE,15,PMDB,PARTIDO DO MOVIMENTO DEMOCRATICO BRASILEIRO,-1,DB,PMDB/PSD
3,21/11/2018,15:22:40,1994,1,ELEICOES 1994,BR,BR,BRASIL,1,PRESIDENTE,...,#NULO,HERNANI FORTUNA,-1,#NE,20,PSC,PARTIDO SOCIAL CRISTAO,-1,#NULO,#NULO
4,21/11/2018,15:22:40,1994,1,ELEICOES 1994,BR,BR,BRASIL,1,PRESIDENTE,...,#NULO,CARLOS GOMES,-1,#NE,36,PRN,PARTIDO DA RECONSTRUCAO NACIONAL,-1,#NULO,#NULO
5,21/11/2018,15:22:40,1994,1,ELEICOES 1994,BR,BR,BRASIL,1,PRESIDENTE,...,#NULO,FERNANDO HENRIQ,-1,#NE,45,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,-1,UTP,PTB/PFL/PSDB
6,21/11/2018,15:22:40,1994,1,ELEICOES 1994,BR,BR,BRASIL,1,PRESIDENTE,...,#NULO,ENEAS,-1,#NE,56,PRONA,PARTIDO DA REEDIFICACAO DA ORDEM NACIONAL,-1,#NULO,#NULO
7,21/11/2018,15:22:40,1994,1,ELEICOES 1994,BR,BR,BRASIL,2,VICE PRESIDENTE,...,#NULO,#NULO,-1,#NE,11,PPR,PARTIDO PROGRESSISTA REFORMADOR,-1,#NULO,#NULO
8,21/11/2018,15:22:40,1994,1,ELEICOES 1994,BR,BR,BRASIL,2,VICE PRESIDENTE,...,#NULO,#NULO,-1,#NE,12,PDT,PARTIDO DEMOCRATICO TRABALHISTA,-1,#NULO,#NULO
9,21/11/2018,15:22:40,1994,1,ELEICOES 1994,BR,BR,BRASIL,2,VICE PRESIDENTE,...,#NULO,#NULO,-1,#NE,13,PT,PARTIDO DOS TRABALHADORES,-1,FBPPC,#NULO


In [74]:
data.columns[20]:data.columns[30]

In [32]:
names = 'DATA_GERACAO HORA_GERACAO ANO_ELEICAO NUMERO_TURNO DESCRICAO_ELEICAO SIGLA_UF SIGLA_UE DESCRICAO_UE CODIGO_CARGO DESCRICAO_CARGO NOME_CANDIDATO SEQUENCIAL_CANDIDATO NUMERO_CANDIDATO NUMERO_CPF_CANDIDATO NOME_URNA_CANDIDATO CODIGO_SITUACAO_CANDIDATURA DESCRICAO_SITUACAO_CANDIDATURA NUMERO_PARTIDO SIGLA_PARTIDO NOME_PARTIDO CODIGO_LEGENDA SIGLA_LEGENDA COMPOSICAO_LEGENDA NOME_LEGENDA CODIGO_OCUPACAO DESCRICAO_OCUPACAO DATA_NASCIMENTO NUMERO_TITULO_ELEITORAL_CANDIDATO IDADE_DATA_ELEICAO CODIGO_SEXO DESCRICAO_SEXO CODIGO_GRAU_INSTRUCAO DESCRICAP_GRAU_INSTRUCAO CODIGO_ESTADO_CIVIL DESCRICAO_ESTADO_CIVIL CODIGO_NACIONALIDADE DESCRICAO_NACIONALIDADE SIGLA_UF_NASCIMENTO CODIGO_MUNICIPIO_NASCIMENTO NOME_MUNICIPIO_NASCIMENTO DESPESA_MAX_CAMPANHA CODIGO_SITUACAO_TOTAL_TURNO DESCRICAO_SITUACAO_TOTAL_TURNO'.split(' ')
len(names)

43